In [22]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import category_encoders as ce

In [2]:
df = pd.read_csv('loan_default.csv')
df.head(3)

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co_applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
1,24932,2019,cf,Male,nopre,type3,p4,l1,nopc,nob/c,...,EQUI,765,EXP,45-54,to_inst,NaN,North,direct,1,NaN
2,25039,2019,cf,Male,nopre,type2,p3,l1,nopc,b/c,...,EQUI,880,EXP,35-44,to_inst,NaN,North,direct,1,NaN


In [6]:
df.Status.value_counts(normalize=True)

Status
0    0.753555
1    0.246445
Name: proportion, dtype: float64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [13]:
X = df.drop(columns=['Status'])
y = df.Status

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [15]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('woe', ce.WOEEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = LogisticRegression(max_iter=500)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

In [16]:
pipeline.fit(X_train, y_train)

C:\Users\carla.scherer\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['ID', 'year', 'loan_amount', 'rate_of_interest', 'Interest_rate_spread',
       'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score',
       'LTV', 'dtir1'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('woe',
                                                                   WOEEncoder())]),
                                                  Index(['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose',
       'Credit_Worthiness', 'open_credit', 'business_or_commercial',
       'Neg_ammortization', 'interest_only', 'lump_sum_payment',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'credit_type', 'co_applicant_credit_type', 'age',
       'submission_of_application', 'Region', 'Security_Type'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=500))])

In [17]:
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.8476827873814489

In [20]:
selector = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=7, step=1)
X_train_transformed = preprocessor.fit_transform(X_train, y_train)
selector.fit(X_train_transformed, y_train)

C:\Users\carla.scherer\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\carla.scherer\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=7)

In [21]:
selected_columns = pd.Series(X_train.columns)[selector.support_]
print(selected_columns.tolist())

['Interest_rate_spread', 'construction_type', 'income', 'credit_type', 'co_applicant_credit_type', 'age', 'Region']


# Treinando o modelo final

In [23]:
X = df.loc[:, selected_columns.to_list()]
y = df.Status

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [27]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('woe', ce.WOEEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = LogisticRegression(max_iter=500)

pipeline_final = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

In [28]:
pipeline_final = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

In [29]:
pipeline_final.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Interest_rate_spread', 'income'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('woe',
                                                                   WOEEncoder())]),
                                                  Index(['construction_type', 'credit_type', 'co_applicant_credit_type', 'age',
       'Region'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=500))])

In [30]:
y_pred = pipeline_final.predict(X_test)
accuracy_score(y_test, y_pred)

0.8553171453554853

In [31]:
filename = 'propensity_model.pkl'
pickle.dump(pipeline_final, open(filename, 'wb'))

# Criando dados para request

In [38]:
X_test.iloc[24914].values

array([1.1478, 'sb', 3180.0, 'CIB', 'CIB', '55-64', 'North'], dtype=object)

In [58]:
request_array = [[1.1478, 'sb', 3180.0, 'CIB', 'CIB', '55-64', 'North']]

In [65]:
selected_columns.to_list()

['Interest_rate_spread',
 'construction_type',
 'income',
 'credit_type',
 'co_applicant_credit_type',
 'age',
 'Region']

In [59]:
request_data = pd.DataFrame(request_array, columns=selected_columns.to_list())
request_data

,Interest_rate_spread,construction_type,income,credit_type,co_applicant_credit_type,age,Region
0,1.1478,sb,3180.0,CIB,CIB,55-64,North


In [60]:
pipeline_final.predict(request_data)

array([0], dtype=int64)

In [66]:
request_sample = request_data.to_json()

with open("docker/request_sample.json", "w") as file:
    file.write(request_sample)

In [67]:
!pip freeze > requirements.txt